In [23]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import tensorflow as tf
# from mylib.PMun_Week3Lec16PP import KernelHyperParameterLearning
# from mylib.PMun_Week3Lec16PP import KernelFunctionWithTensorflow


# 미완성 코드임.
# 차 후 Hyperparameter Learning for Gaussian Process 미션에서 뼈대로 쓰일 수 있다.


In [24]:

def KernelFunctionWithTensorflow(theta0, theta1, theta2, theta3, X1,X2):
    insideexp1 = tf.mul(tf.div(theta1, 2.0), np.dot((X1-X2), (X1-X2)))
    insideexp2 = theta2
    insideexp3 = tf.mul(theta3, np.dot(np.transpose(X1), X2))
    insideexp = tf.add(tf.add(insideexp1, insideexp2),insideexp3)
    ret = tf.mul(theta0, tf.exp(insideexp))
    return ret


In [25]:

def KernelHyperParameterLearning(iteration, learningRate, trainingX, trainingY):
    numDataPoints = len(trainingY)
    numDimension = len(trainingX[0])
 
    # Input and Output Declaration for tensorflow
    obsX = tf.placeholder(tf.float32, [numDataPoints,numdimension])
    obsY = tf.placeholder(tf.float32, [numDataPoints, 1])
 
    # Learning Parameter Variable Declaration for tensorflow
    theta0 = tf.Variable(1.0)
    theta1 = tf.Variable(1.0)
    theta2 = tf.Variable(1.0)
    theta3 = tf.Variable(1.0)
    beta = tf.Variable(10.0)
 
    # Kernel building
    matCovarianceLinear = []
    for i in range(numDataPoints):
        for j in range(numDataPoints):
            kernelEvaluationResult =kernelFunctionWithTensorflow(theta0, theta1,theta2, theta3, 
                                                                 tf.slice(obsX, [i,0], [1,numdimension]), 
                                                                 tf.slice(obsX, [j,0], [1,numDimension]))
            if i != j:
                matCovarianceLinear.append(kernelEvaluationResult)
            if i == j:
                matCovarianceLinear.append(kernelEvaluationResult + tf.div(1.0,beta))
    matCovarianceCombined =tf.pack(matCovarianceLinear)
    matCovariance =tf.reshape(matCovarianceCombined, [numDataPoints, numDataPoints])
    matCovarianceInv =tf.inv(matCovariance)
    

In [27]:

# Prediction
negloglikelihood = 0.0
for i in range(numDataPoints):
    k = tf.Variable(tf.ones([numDataPoints]))
    for j in range(numDataPoints):
        kernelEvaluationResult = kernelFunctionWithTensorflow(theta0, theta1,theta2, theta3, 
                                                             tf.slice(obsX, [i,0], [1,numdimension]), 
                                                             tf.slice(obsX, [j,0], [1,numDimension]))
        indices = tf.constant([j])
        tempTensor = tf.Variable(tf.zeros([1]))
        tempTensor = tf.add(tempTensor,kernelEvaluationResult)
        tf.scatter_update(k, tf.reshape(indices, [1,1]),tempTensor)
                    
    c = tf.Variable(tf.zeros([1,1]))
    kernelEvaluationResult = kernelFunctionWithTensorflow(theta0, theta1,theta2, theta3, 
                                                          tf.slice(obsX, [i,0], [1,numdimension]), 
                                                          tf.slice(obsX, [j,0], [1,numDimension]))
    c = tf.div(tf.add(tf.add(c, kernelEvaluationResult), 1), beta)
    
    k = tf.reshape(k, [1, numDataPoints])
        
    predictionMu = tf.matmul(k, tf.matmul(matCovarianceInv, obsY))
    predictionVar = tf.sub(c, tf.matmul(k,tf.matmul(matCovarianceInv, tf.transpose(k))))
        
    negloglikelihood = tf.add(negloglikelihood, tf.div(tf.pow(tf.sub(predictionMu,tf.slice(obsY, [i,0], [1,1])), 2), 
                                                       tf.scalar_mul(tf.constant(2,0),predictionVar)))

# Training session declaration
training=tf.train.GradientDescentOptimizer(learningRate).ninimize(negloglikelihood)


NameError: name 'learningRate' is not defined